# HPC

**Table of contents**
* [Pending classes to review](#Pending-classes-to-review)
* [Class 008 - Parallel code scalability](#Class-008-Parallel-code-scalability)
  * [Synchronicity](#Synchronicity)
  * [Flynn's taxonomy](#Flynn's-taxonomy)
  * [Types of memory systems](#Types-of-memory-systems)
  * [Speedup and efficiency](#Speedup-and-efficiency)
  * [Amdahl's Law](#Amdahl's-Law)
  * [Gustafon's Law](#Gustafson's-Law)
  * [Weak scalability vs. Strong scalability](#Weak-scalability-vs.-Strong-scalability)


## Pending classes to review

* Class 007 - Code Profiling
* Class 006 - Measuring time, experimental errors and reproducibility
* Class 005 - Git, Github and TravisCI
* Class 004 - Reproducibility, compilers and autobuilders
* Class 003 - Containers
* Class 002 - History of processors development

## Class 008 - Parallel code scalability

> **Date:** 2021-04-17  
> **Subjects:**
> * [Synchronicity](#Synchronicity)
> * [Flynn's taxonomy](#Flynn's-taxonomy)
> * [Types of memory systems](#Types-of-memory-systems)
> * [Speedup and efficiency](#Speedup-and-efficiency)
> * [Amdahl's Law](#Amdahl's-Law)
> * [Gustafon's Law](#Gustafson's-Law)
> * [Weak scalability vs. Strong scalability](#Weak-scalability-vs.-Strong-scalability)
> 
> **References:**
> * [Recorded class - First part][class-recording-01]
> * [Recorded class - Second part][class-recording-02]
> * [Slides][class-slides]

[class-recording-01]: https://drive.google.com/file/d/1Eqq6bTxox54uH1siH--NqDBNnSWvxvEi/view?usp=sharing
[class-recording-02]: https://drive.google.com/file/d/1RzKEi6CAXYNo-GfJ63VjQSm5SgzAfBzX/view?usp=sharing
[class-slides]: https://drive.google.com/file/d/1I5Vkz9oI0KCFGw-YgP-B1G8TFz88vTqB/view?usp=sharing

### Synchronicity

While executing a given program, you can have two types of operations:

1. **Synchronous operations:** These operations are those that can happen at the same time. So, they can be parallelized
2. **Asynchronous operations:** These operations are not happening at the same time

During the execution of a given program, we can have some synchronization operations (like a **barrier operation**) that could synchronize our code for us. These are extremelly helpful while coding a parallelizable system, because it is common that some operations can happen synchronously, but other don't.

The barrier operations can synchrone multiple **threads** for the same point. For example, we can have the following operations:

$$ T_1 \longrightarrow T_3 \longrightarrow T_5$$

If the $T_5$ operations depends on the $T_3$ result, we need to add a barrier before that if we want to parallelize our code. Now we have:

$$ T_1 \longrightarrow T_3 \longrightarrow B \longrightarrow T_5$$

Being $B$ the barrier operation.

Now, we can safely execute our code.

### Flynn's taxonomy

The **Flynn's taxonomy** aims to standardize the possible scenarios of a given code execution. It has three types os execution scenarios:

#### 1. SISD (Single instruction stream, single data stream)

This is the classical, von Neumann, scenario. Here, we have a sequential execution, where only a single process is being executed and receiving inputs from a single data stream.

<img src="./assets/hpc-sisd-diagram.png" width="250px" />

#### 2. SIMD (Single instruction stream, multiple data stream)

In this scenario, we have multiple operations being execution for each instruction.

<img src="./assets/hpc-simd-diagram.png" />

#### 3. MISD (Multiple instruction stream, single data stream)

In this scenario, we have just a single data stream, but being processed by multiple instruction streams

<img src="./assets/hpc-misd-diagram.png" width="250px" />

#### 4. MIMD (Multiple instruction stream, multiple data stream)

In this scenario we have both data and instruction flow independents. So, we have multiple data streams, and also multiple threads executing those.

<img src="./assets/hpc-mimd-diagram.png" width="250px" />

### Types of memory systems

In a given application, we can have two types of memory systems:

1. **Shared-memory system:** This happens when all proccess in that program are sharing the same memory. Usually, when we want to parallelize these programs, we need to use a given API like [OpenMP](https://www.openmp.org/)
2. **Distributive-memory systems:** These type of systems are when your proccesses are running in different memory points. *Usually* this happens when they are phisically separated (different nodes), **but that is not always the case**. Some computers can have separate memory points for different instances of a given program *in the same node*. In this type of system, the **network** is extremelly important. Because we need to use a message API (like [MPI](https://www.open-mpi.org)) to synchronize our nodes.

One important thing to note is **latency**. When we have the second system type, we need to watch out for latency. So, we need a good performance in our network layer.

Based on this, we've created the **BSP (Bulk synchronous parallel) programming model**. That model aimed to solve how we code synchronous systems, by creating a standardized way to do so. This was introduced [in an article](https://dl.acm.org/doi/10.1145/79173.79181). That model is organized in **rounds** where, in each round, all processes do the following:

1. Do the computation locally
2. Send messages for all other processes
3. Way all messages to be received, waiting a am given barrier until the next round, when all proccesses start together

Here, we have two important keywords to note:

* **Overhead:** Which is the excess or indirect computation time, memory, or anything that are required to perform a specific task
* **Latency:** Which is the amount of time waiting our network
* **Bottlenecks:** Which is basically which part of our system is overloaded

There are multiple types of bottlenecks, some common are:

* **IO bound:** When we can't increase our performance, since our IO device is overloaded
* **CPU bound:** When we can't increase our performance, since our CPU is overloaded
* **Memory bound:** Same as above

Basically, we can have *anything* bound as a type of bottleneck.

### Speedup and efficiency

#### Speedup

It is a metric that measures the increase of performance for a given proccess when we parallelize it. Usually it uses the following notations:

$$
T_{s} = \text{Serial (not parallel) execution time} \\
T_{p} = \text{Parallel execution time}
S = \text{Speedup}
$$

We can calculate the speedup with following formula:

$$
S = \dfrac{T_s}{T_p}
$$

#### Efficiency

Measures the efficiency of our parallel proccess. It is calculated by the following formula:

$$
E = \dfrac{S(p)}{p}
$$

Where:

* $E$: Efficiency
* $S$: Speedup
* $p$: Amount of parallel resources

Based on this, we can state that it is possible to have a **linear speedup**. We can state that when this statement is `true`:

$$
S(p) = 1 \land E = 1
$$

In this scenario (both speedup and effiency are $1$), we can say that we can decrease our execution time at the same rate as we add new parallel resources

### Amdahl's Law

This is a law that was created based on the observations of [this article](https://dl.acm.org/doi/10.1145/1465482.1465560). That law calculates the parallel execution time of a given process.

Considering:
* $T_p$ = The parallel execution time
* $T_s$ = The serial execution time
* $r$ = The fraction of the program that can be parallelized
* $p$ = Number of parallel resources

This law states that:
$$
T_p = (1 - r) \times T_s + \dfrac{(r \times T_s)}{p}
$$

So, when we calculate the limit of that execution time:
$$
\lim_{p \to +\infty} T_P = (1-r) \times T_s
$$

Passing it to the `Speedup` and `Efficiency`:

$$
\lim_{p \to +\infty} S = \dfrac{T_s}{(1 - r) \times T_s} = \dfrac{1}{1 - r} \\
$$
$$
\lim_{p \to +\infty} E = 0
$$

Based on this law, we can state that the maximum performance of a given program drastically changes based on how much that program is parallelizable. We can have a lot of examples like the following:

* If a given program is 10% serial
* Our max speedup is 10 times

So, in this case, the maximum performance is 10 times. We usually consider this as a hard definition, since it doesn't consider the problem issue.

### Gustafson's Law

This law was proposed in [this article](https://dl.acm.org/doi/10.1145/42411.42415), where Gustavson analyses the Amdahl's Law, and states that it simply ignores the size of the problem our program is dealing with. In a nutshell, it highlights that Amdahl's Law compares the execution time of the same algorithm using the same problem size, comparing different amount of parallel resources, but even if Amdahl's Law states that a given number is our maximum speedup, that number could be bigger if we increase the size of our problem.

<img src="./assets/hpc-amdahl-vs-gutafson.gif" />

In Gustafson's Law, we consider the parallel portion of the code as $r$ and we multiple that $r$ amount in our serial comparisson by the number of parallel resources, for example:

<img src="./assets/hpc-gustafson-graphical-representation.png" />

So, for Gutafson, we have the following:

$$
S = 1 - r + r \times p \\
E = \dfrac{r + (1 - r)}{p}
$$

Where:

* $S$ = Scaled speedup
* $r$ = Percentual portion of the program that can be parallelized
* $p$ = Amount of parallel resources
* $E$ = Efficiency

If we analyse it in infinity, we have:

$$
\lim_{p \to +\infty} S = r \times p \\
\lim_{p \to +\infty} E = \sim r
$$

### Weak scalability vs. Strong scalability

Scalability is an extremelly common topic in computer science. We can use two main sources to define it:
* [This article](http://pages.cs.wisc.edu/~markhill/papers/can90_scalability.pdf)
* [This book](https://www.amazon.com/Introduction-Parallel-Programming-Peter-Pacheco/dp/0123742609)

Those sources define a scalable software as:
> A scalable software can increase the amount of parallel resources to increase performance at the same rate.

If the increase in the number of parallel resources has a relation with the increase of the problem with a constant efficiency, we can state that the program is scalable. Here is a simple example:

<img src="./assets/hpc-scalable-chart-example.png" />

We can define two main types of scalability:

1. **Strong scalability:** This happens when we keep the efficiency when we increase the parallelism without changing the problem size (Amdahl's based)
2. **Weak scalability:** This happens when we keep the efficiency when we increase the parallelism and the problem size at the same rate (Gustafson's based)